In [35]:
# download the dataset and save it in the same dir with this file.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel,RationalQuadratic,Exponentiation
from sklearn.tree import DecisionTreeRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
import os
kf = KFold(n_splits=10)
# import scikit learn
from sklearn import linear_model,svm
from sklearn.preprocessing import PolynomialFeatures


In [36]:
def getxandy(filename):
  with open(filename,'r') as f:
    df = pd.read_csv(f,index_col=0)
    x = np.zeros((len(df.x),2))
    y = np.zeros((len(df.x)))
    x[:,0] = df.x.to_numpy()
    x[:,1] = df.y.to_numpy()
    y = df.z.to_numpy()
  return x,y

In [37]:
def getpolyxandy(filename,poly=2):
  with open(filename,'r') as f:
    df = pd.read_csv(f,index_col=0)
    x = np.zeros((len(df.x),2))
    y = np.zeros((len(df.x)))
    x[:,0] = df.x.to_numpy()
    x[:,1] = df.y.to_numpy()
    y = df.z.to_numpy()
    poly = PolynomialFeatures(degree=poly)
    x = poly.fit_transform(x)
  return x,y

In [38]:
def scoreperdataset(y_pred,y_test):
  return np.sqrt(np.sum(np.square(y_pred - y_test))/len(y_pred))

In [39]:
def writeresults(X_test,y_test,y_pred,f):
    idx = np.argsort(X_test[:,0])
    X_test = X_test[idx]
    y_test = y_test[idx]
    y_pred = y_pred[idx]
    data = {}
    data['x'] = X_test[:,0]
    data['y'] = X_test[:,1]
    data['z_test'] = y_test[idx]
    data['z_pred'] = y_pred[idx]
    df = pd.DataFrame(data)
    if not os.path.isdir('results/'):
      os.mkdir('results')
    df.to_csv('results/'+f+'.csv',float_format="%.6f")

In [47]:
# pre 9 large

#model = Pipeline([('poly', PolynomialFeatures(degree=4)),('linear', DecisionTreeRegressor(max_depth=25))])

finalacc_large = []
trainfolder = 'large_datasets'
datasets = [chr(ord('A') + x) for x in range(18)]
for d in datasets:
  X,y = getpolyxandy(trainfolder+'/'+ d +'/train.csv',4)
  reg = DecisionTreeRegressor(max_depth=25)
  X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.1)
  reg.fit(X_train,y_train)
  y_pred = reg.predict(X_test)
  loss_large = scoreperdataset(y_pred, y_test)
  finalacc_large.append(loss)
  print("{} loss {:.5f}".format(d, loss_large))
  #writeresults(X_test,y_test,y_pred)
print('final average loss is {:.5f}'.format( sum(finalacc_large)/len(finalacc_large) ) )

KeyboardInterrupt: 

In [55]:
# pre 9
finalacc_small = []
trainfolder = 'small_datasets'
datasets = [chr(ord('A') + x) for x in range(18)]
for d in datasets[:9]:
  X,y = getpolyxandy(trainfolder+'/'+ d +'/train.csv',4)
  reg = DecisionTreeRegressor(max_depth=25)
  X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.1)
  reg.fit(X_train,y_train)
  y_pred = reg.predict(X_test)
  loss_small = scoreperdataset(y_pred, y_test)
  finalacc_small.append(loss_small)
  print("{} loss {:.5f} ".format(d, loss_small))
  #writeresults(X_test,y_test,y_pred,f[9:])
print('final avg loss is {:.5f}'.format(sum(finalacc_small)/len(finalacc_small)))

A loss 0.12645 
B loss 0.17270 
C loss 0.41336 
D loss 0.03651 
E loss 0.05273 
F loss 0.22237 
G loss 0.01887 
H loss 0.12639 
I loss 0.22098 
final avg loss is 0.15448


In [58]:
# pre 9
finalacc_small = []
trainfolder = 'small_datasets'
datasets = [chr(ord('A') + x) for x in range(18)]
for d in datasets[9:]:
  X,y = getpolyxandy(trainfolder+'/'+ d +'/train.csv',4)
  reg = DecisionTreeRegressor(max_depth=25)
  X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.1)
  reg.fit(X_train,y_train)
  y_pred = reg.predict(X_test)
  loss_small = scoreperdataset(y_pred, y_test)
  finalacc_small.append(loss_small)
  print("{} loss {:.5f} ".format(d, loss_small))
  #writeresults(X_test,y_test,y_pred,f[9:])
print('final avg loss is {:.5f}'.format(sum(finalacc_small)/len(finalacc_small)))

J loss 0.74384 
K loss 0.74708 
L loss 0.84819 
M loss 0.73457 
N loss 0.72024 
O loss 0.76775 
P loss 0.70474 
Q loss 0.73537 
R loss 0.58388 
final avg loss is 0.73174


In [78]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
final_acc = []
pred_array = {}
for d in datasets:
  X,y = getpolyxandy('small_datasets/'+'R'+'/train.csv',4)
  clf = GridSearchCV(DecisionTreeRegressor(), {'max_depth':[10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30]})
  X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.1, random_state=1)
  clf.fit(X_train, y_train)
  y_pred = clf.best_estimator_.predict(X_test)
  loss = scoreperdataset(y_pred,y_test)
  final_acc.append(loss)
  X_real,y_real = getpolyxandy('small_datasets/'+'R'+'/train.csv',4)
  y_real_pred = clf.best_estimator_.predict(X_real)
  pred_array[d+'.z'] = y_real_pred
  print('loss for f is {}, maxdepth is {}'.format(loss, clf.best_estimator_.max_depth))
print('final avg loss is {:.5f}'.format( sum(final_acc)/len(final_acc)))
outdf = pd.DataFrame(pred_array)
outdf.to_csv('submission.csv',index_label='id')

final avg loss is 0.57828
